# Bridging Neural Dynamics: extending TRFs - Part 2 

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Hugo-W/CNSP25_tutorial_HWeissbart/main/)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hugo-W/CNSP25_tutorial_HWeissbart/blob/main/notebooks/part_2.ipynb)


This time let's load some real MEG data and try our freshly designed feature-based Phase-Amplitude Coupling extraction. If time allows, will move onto trying to disentangle brain response to acoustic edges around uncertain vs more certain words, to see whether top-down linguistic information affects lower level envelope tracking.

### Setup (Google Colab only)

> We need an extra step to make sure widgets can work in Collab notebooks, uncomment and run the cell bellow if using Google Collab, otherwise just skip this.

In [ ]:
try:
  import google.colab # type: ignore
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    %pip install -q ipywidgets ipympl
    %pip install -q natmeeg

In [ ]:
import sys
if IN_COLAB and 'natmeeg' not in sys.modules:
    print("🔁 Restarting kernel to use newly installed packages...")
    get_ipython().kernel.do_shutdown(True) # type: ignore

In [ ]:
# Download `utils`
if IN_COLAB:
    import requests
    import os

    files = ['__init__.py', 'helpers.py', 'surrogate.py', 'visu.py', 'utils.py']
    # Create utils directory if it doesn't exist
    os.makedirs('utils', exist_ok=True)
    for f in files:
        url = f'https://raw.githubusercontent.com/Hugo-W/CNSP25_tutorial_HWeissbart/main/utils/{f}'
        response = requests.get(url)

        with open(f'utils/{f}', 'wb') as f:
            f.write(response.content)

    from google.colab import output # type: ignore
    output.enable_custom_widget_manager()

## Imports and Data download

First let's get all the libraries, data and path setup.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import VBox, HBox, HTML, HTMLMath, Checkbox, FloatSlider, IntSlider, Dropdown, interact, Button
import tqdm.notebook as tqdm
import sys, os
# Add utils to Python path
sys.path.append('..') # access to utils.py and visu.py modules
from utils.utils import download_file, list_h5_data
from utils.visu import plot_fft
from utils.surrogate import (simulate_background_eeg, simulate_spike_events,
                             simulate_smooth_signal, create_kernel)
from utils.helpers import lowpass_filter, convolve_with_kernel, lag_matrix, lag_span, _svd_regress, svd_trf_estimation

The cell below will download the data if they are not already present in the `data/` folder.

In [2]:
data_dir = "../data/"
files = {
    "sub-001/audioBook-filtered-ICAed-raw.fif": "https://osf.io/download/689ca8ab66c0f8121bd5a0fd/",
    "sub-001/audioBook-eve.fif": "https://osf.io/download/d7phu/",
    "sub-001/noise-cov.fif": "https://osf.io/download/689ca6fa89cdf5804c13c704/",
    "stim/predictors.hdf5": "https://osf.io/download/68b3114c95a919eba5fe1c79/"
}
# Bad channels: MLF12, MRC55, MRF61

for file_name, url in files.items():
    file_path = os.path.join(data_dir, file_name)
    # Check if the data directory exists, if not create it
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    # Check if the file already exists
    if not os.path.exists(file_path):
        print(f"{file_name} not found. Downloading... (this operation may take a couple of minutes depending on your connection)")
        download_file(url, file_path)
    else:
        print(f" ✅ File {file_name} already exists in the {data_dir} directory.")

 ✅ File audioBook-filtered-ICAed-raw.fif already exists in the ../data/sub-001 directory.
 ✅ File audioBook-eve.fif already exists in the ../data/sub-001 directory.
 ✅ File noise-cov.fif already exists in the ../data/sub-001 directory.


Let's have a look into the HDF5 file of stimulus features:

In [ ]:
list_h5_data("../data/stim/predictors.hdf5", max_depth=4)

transcripts_v2
├── 100
│   ├── acoustic
│   │   ├── ANGE_part1_normalized (27402,)
│   │   ├── Anderson_S01_P01_normalized (29800,)
│   │   ├── Anderson_S01_P02_normalized (31770,)
│   │   ├── Anderson_S01_P03_normalized (28900,)
│   │   ├── Anderson_S01_P04_normalized (35080,)
│   │   ├── BALL_part1_normalized (29801,)
│   │   ├── EAUV_part1_normalized (34382,)
│   │   ├── EAUV_part2_normalized (36153,)
│   │   ├── grimm_20_1_normalized (36659,)
│   │   ├── grimm_20_2_normalized (40055,)
│   │   ├── grimm_23_1_normalized (30309,)
│   │   ├── grimm_23_2_normalized (33287,)
│   │   ├── grimm_23_3_normalized (30292,)
│   ├── wordlevel
│   │   ├── ANGE_part1_normalized (27402, 9)
│   │   ├── Anderson_S01_P01_normalized (29800, 9)
│   │   ├── Anderson_S01_P02_normalized (31770, 9)
│   │   ├── Anderson_S01_P03_normalized (28900, 9)
│   │   ├── Anderson_S01_P04_normalized (35080, 9)
│   │   ├── BALL_part1_normalized (29801, 9)
│   │   ├── EAUV_part1_normalized (34382, 9)
│   │   ├── EAUV_par

We can see that the stimulus feature file `predictors.hdf5` contains a matrix of feature per story, organised under two keys: `acoustic` and `wordlevel`. The former contains the smooth continuous feature (only envelop here), while the latter contains impulse-like time series of word-level features. 

We can also observe that word level features are matrices with 9 columns. These map to the following features:
- Word Onsets
- Word frequency
- Surprisal
- Entropy
- KL divergence
- Prediction error (surprisal weighted by precision, with precision ~1/entropy)
- Depth
- Close
- Open